In [2]:
import pandas as pd
import numpy as np
import os

## Cleaning CalRecycle Waste Data

### Combine all individual county files into a single df

In [2]:
def filter_waste_origin_df(df, county):
    """Drop quarterly values in the csv and only keep yearly values
    Input:
        - calrecycle df
        - county (string)
    
    Ouput:
        - df with year, county, disposal ton
    """
    columns_to_keep = ["Report Year", "Diposal Ton", "County"]
    df["County"] = county
    missing_data = df["Report Year"].isnull()
    df = df[~missing_data]
    return df[columns_to_keep]

In [3]:
DATA_PATH = "../calrecycle-data/"
county_waste_production = []
for f in os.listdir(DATA_PATH):
    filename, ext = os.path.splitext(f)
    county = filename.split("_waste_production")[0]
    county = county.replace("_", " ")
    print("[INFO] filtering county file: ", os.path.join(DATA_PATH, f))
    try:
        df = pd.read_excel(os.path.join(DATA_PATH, f), skiprows=3, skipfooter=2)
        filtered_df = filter_waste_origin_df(df, county)
        county_waste_production.append(filtered_df)
    except Exception as e:
        print("[DEBUG] Skipping: ", county)

county_waste_production_df = pd.concat(county_waste_production)

[INFO] filtering county file:  ../calrecycle-data/San_Bernardino_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Tehama_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Modoc_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/San_Benito_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Santa_Barbara_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Yolo_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Sonoma_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Tuolumne_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/San_Mateo_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Monterey_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Alpine_waste_production.xlsx
[INFO] filtering county file:  ../calrecycle-data/Solano_waste_production.xlsx
[INFO] filtering county file:

In [4]:
county_waste_production_df.rename(columns={"Report Year": "Year", 
                                           "Diposal Ton": "Waste Produced (Tons)"}, inplace=True)
county_waste_production_df.head()

,Year,Waste Produced (Tons),County
0,1995.0,1623266.99,San Bernardino
105,1996.0,1652495.58,San Bernardino
210,1997.0,1603700.60,San Bernardino
315,1998.0,1682447.54,San Bernardino
420,1999.0,1679816.31,San Bernardino


### Tests - sanity check that nothing went wrong during data cleaning

- Total number of counties (excluding Sutter) * total number of years = 58 - 1 * (2019 - 1995 + 1) = total number of observations in the final df
- Alameda County 2019 = 1,465,264	
- LA County 2012 total waste produced = 8,141,712
- Orange County 2000 total waste produced = 3,803,041

In [6]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert county_waste_production_df.shape[0] == (58 - 1) * (2019 - 1995 + 1)
assert np.isclose(1465264, get_feature(county_waste_production_df, "Alameda", 2019, "Waste Produced (Tons)"))
assert np.isclose(8141712, get_feature(county_waste_production_df, "Los Angeles", 2012, "Waste Produced (Tons)"))
assert np.isclose(3803041, get_feature(county_waste_production_df, "Orange", 2005, "Waste Produced (Tons)"))

### Save cleaned waste production csv after passing tests

In [7]:
county_waste_production_df.to_csv("../data/calrecycle_waste_production_per_county.csv", index=False)

In [8]:
# def get_feature(df, feature, **kwargs):
#     """Gets a specific feature from the dataframe
#     """
#     year = kwargs["year"]
#     county = kwargs["county"]
#     feature = kwargs["feature"]
#     county_condition = df["County"] == county
#     year_condition = df["Year"] == year
#     return df.loc[county_condition & year_condition, feature]

# get_feature(county_waste_production_df, **{"feature": "Waste Produced (Tons)", "year": 2012, "county":"Los Angeles"})

---

## Cleaning Census Population Data

### 2010-2019

In [9]:
county_population_2010_2019 = pd.read_excel("../data/ca-finance-dpt-county-populations-2010-2019.xlsx", 
                                            sheet_name="Table 1 County State", header=1, skipfooter=1)
county_population_2010_2019.rename(columns={"COUNTY":"County"}, inplace=True)
county_population_2010_2019["County"] = county_population_2010_2019.County.str.strip()
county_population_2010_2019.head()

,County,4/1/2010,1/1/2011,1/1/2012,1/1/2013,1/1/2014,1/1/2015,1/1/2016,1/1/2017,1/1/2018,1/1/2019,1/1/2020
0,Alameda,1510271,1526110,1545917,1569989,1590729,1613528,1632599,1646711,1655306,1664783,1670834
1,Alpine,1175,1169,1166,1164,1163,1162,1162,1161,1159,1149,1142
2,Amador,38091,37173,36777,36267,36029,36114,36055,36928,37563,37820,37676
3,Butte,220000,220620,221340,222374,222988,223940,224096,225643,226374,221521,210291
4,Calaveras,45578,45540,45496,45424,45358,45265,45244,45170,45155,45085,45023


In [10]:
county_population_2010_2019.set_index("County", inplace=True)
col_names_date = county_population_2010_2019.columns 
col_names_year = [int(date[-4:]) for date in col_names_date]
county_population_2010_2019.columns = col_names_year
county_population_2010_2019.reset_index(inplace=True)
county_population_2010_2019.head()

,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alameda,1510271,1526110,1545917,1569989,1590729,1613528,1632599,1646711,1655306,1664783,1670834
1,Alpine,1175,1169,1166,1164,1163,1162,1162,1161,1159,1149,1142
2,Amador,38091,37173,36777,36267,36029,36114,36055,36928,37563,37820,37676
3,Butte,220000,220620,221340,222374,222988,223940,224096,225643,226374,221521,210291
4,Calaveras,45578,45540,45496,45424,45358,45265,45244,45170,45155,45085,45023


In [11]:
county_population_2010_2019 = county_population_2010_2019.melt(id_vars="County", var_name='Year', value_name="Population")
county_population_2010_2019.head()

,County,Year,Population
0,Alameda,2010,1510271
1,Alpine,2010,1175
2,Amador,2010,38091
3,Butte,2010,220000
4,Calaveras,2010,45578


### 2000-2009

In [12]:
county_population_2000_2009 = pd.read_excel("../data/ca-finance-dpt-county-populations-2000-2009.xls", 
                                            sheet_name="Table 1 County State", 
                                            header=3, skipfooter=2)
county_population_2000_2009.rename(columns={"COUNTY":"County"}, inplace=True)
county_population_2000_2009["County"] = county_population_2000_2009.County.str.strip()
county_population_2000_2009.dropna(how='all', inplace=True)
county_population_2000_2009.head()

,County,2000-04-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2010-04-01 00:00:00
1,Alameda,1443939.0,1457185.0,1467063.0,1467892.0,1466407.0,1462736.0,1462371.0,1470622.0,1484085.0,1497799.0,1509240.0,1510271.0
2,Alpine,1208.0,1220.0,1242.0,1248.0,1257.0,1237.0,1232.0,1252.0,1228.0,1194.0,1178.0,1175.0
3,Amador,35100.0,35495.0,36159.0,36656.0,36961.0,37434.0,37843.0,38025.0,37975.0,37884.0,38011.0,38091.0
4,Butte,203171.0,204591.0,206942.0,209389.0,211419.0,212955.0,214690.0,216401.0,217801.0,218887.0,219967.0,220000.0
5,Calaveras,40554.0,41042.0,41773.0,42651.0,43554.0,44348.0,45044.0,45477.0,45670.0,45632.0,45602.0,45578.0


In [13]:
county_population_2000_2009.set_index("County", inplace=True)
col_names_date = county_population_2000_2009.columns 
col_names_year = [date.year for date in col_names_date]
county_population_2000_2009.columns = col_names_year
county_population_2000_2009.reset_index(inplace=True)
county_population_2000_2009.drop(columns=[2010], inplace=True) # drop 2010 bc already included in 2010-2019
county_population_2000_2009.head()

,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alameda,1443939.0,1457185.0,1467063.0,1467892.0,1466407.0,1462736.0,1462371.0,1470622.0,1484085.0,1497799.0
1,Alpine,1208.0,1220.0,1242.0,1248.0,1257.0,1237.0,1232.0,1252.0,1228.0,1194.0
2,Amador,35100.0,35495.0,36159.0,36656.0,36961.0,37434.0,37843.0,38025.0,37975.0,37884.0
3,Butte,203171.0,204591.0,206942.0,209389.0,211419.0,212955.0,214690.0,216401.0,217801.0,218887.0
4,Calaveras,40554.0,41042.0,41773.0,42651.0,43554.0,44348.0,45044.0,45477.0,45670.0,45632.0


In [14]:
county_population_2000_2009 = county_population_2000_2009.melt(id_vars="County", var_name='Year', value_name="Population")
county_population_2000_2009.head()

,County,Year,Population
0,Alameda,2000,1443939.0
1,Alpine,2000,1208.0
2,Amador,2000,35100.0
3,Butte,2000,203171.0
4,Calaveras,2000,40554.0


### append 2000-2009 & 2010-2019

In [15]:
county_population_df = county_population_2000_2009.append(county_population_2010_2019)
county_population_df.reset_index(inplace=True, drop=True)
county_population_df.head()

,County,Year,Population
0,Alameda,2000,1443939.0
1,Alpine,2000,1208.0
2,Amador,2000,35100.0
3,Butte,2000,203171.0
4,Calaveras,2000,40554.0


In [16]:
county_population_df.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=object)

### Tests

- Alameda County 2019 population = 1,664,783 
- LA County 2012 population = 9,956,882 
- Orange County 2000 population = 2,956,847

In [8]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert np.isclose(1664783, get_feature(county_population_df, "Alameda", 2019, "Population"))
assert np.isclose(9956882, get_feature(county_population_df, "Los Angeles", 2012,  "Population"))
assert np.isclose(2956847, get_feature(county_population_df, "Orange", 2005, "Population"))

NameError: name 'county_population_df' is not defined

### Save cleaned population csv after passing tests

In [18]:
county_population_df.to_csv("../data/ca-finance-dpt-county-populations-2000-2020.csv", index=False)

---

## Cleaning electricity usage data

In [26]:
electricity_usage_per_county = pd.read_csv("../data/cec-electricity-by-county.csv")
electricity_usage_per_county = electricity_usage_per_county[electricity_usage_per_county.Sector == "Residential"]
electricity_usage_per_county["County"] = electricity_usage_per_county.County.str.lower().str.title()
electricity_usage_per_county.drop(columns=["Total Usage", "Sector"], inplace=True)
electricity_usage_per_county.head()

,County,2019,2018,2017,2016,2015,2014,2013,2012,2011,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
1,Alameda,3064.781376,2945.804917,2991.794416,2917.342741,2893.676837,2891.232325,2984.213677,2998.246885,3023.830340,...,2890.765488,2791.331196,2674.834394,2615.308775,2548.457958,2774.566317,2529.229286,2465.000074,2515.209970,2498.265626
4,Alpine,10.131788,9.420910,9.951697,9.290913,11.855504,11.974975,13.424431,9.298178,9.093424,...,6.597345,6.792005,6.373298,6.063832,6.231561,6.183168,6.812776,5.808735,6.165242,6.036978
7,Amador,140.689119,136.895373,143.509418,135.516470,132.791638,134.744387,139.472718,141.054038,140.216069,...,124.602037,123.223191,114.815322,115.017917,110.744724,110.372675,109.035503,105.907059,108.875556,105.093388
10,Butte,669.328499,717.743260,760.229021,721.205754,712.388497,718.947064,729.845116,738.780910,725.656559,...,629.147565,616.781540,597.449897,594.667475,559.101202,556.276390,551.043168,555.149304,553.073852,547.901570
13,Calaveras,205.365353,201.833277,214.306906,197.961265,191.155224,192.543967,199.732952,201.495546,200.495354,...,170.901166,167.369797,157.714415,153.829455,148.585115,146.123388,145.291775,140.066797,143.201069,138.299372


In [27]:
electricity_usage_per_county = electricity_usage_per_county.melt(id_vars="County", var_name='Year', value_name="Electricity Usage (GWh)")
electricity_usage_per_county["Year"] = pd.to_numeric(electricity_usage_per_county["Year"])
electricity_usage_per_county.head()

,County,Year,Electricity Usage (GWh)
0,Alameda,2019,3064.781376
1,Alpine,2019,10.131788
2,Amador,2019,140.689119
3,Butte,2019,669.328499
4,Calaveras,2019,205.365353


In [30]:
def get_feature(df, county, year, feature):
    """Testing function that gets a specific feature value from the dataframe from a specific year and county
    """
    county_condition = df["County"] == county
    year_condition = df["Year"] == year
    return df.loc[county_condition & year_condition, feature]

assert np.isclose(3064.781376, get_feature(electricity_usage_per_county, "Alameda", 2019, "Electricity Usage (GWh)"))
assert np.isclose(21076.22492, get_feature(electricity_usage_per_county, "Los Angeles", 2012,  "Electricity Usage (GWh)"))
assert np.isclose(6717.053806, get_feature(electricity_usage_per_county, "Orange", 2005, "Electricity Usage (GWh)"))


---

# Merging all individual feature dfs

- add additional features in this section

## Load in all the CSVs

In [19]:
county_waste_production = pd.read_csv("../data/calrecycle_waste_production_per_county.csv")
county_waste_production.head(3)

,Year,Waste Produced (Tons),County
0,1995.0,1623266.99,San Bernardino
1,1996.0,1652495.58,San Bernardino
2,1997.0,1603700.60,San Bernardino


In [20]:
county_population = pd.read_csv("../data/ca-finance-dpt-county-populations-2000-2020.csv")
county_population.head(3)

,County,Year,Population
0,Alameda,2000,1443939.0
1,Alpine,2000,1208.0
2,Amador,2000,35100.0


In [21]:
complete_feature_df = county_waste_production.merge(county_population, on=["County", "Year"], how="inner")
complete_feature_df.head()

,Year,Waste Produced (Tons),County,Population
0,2000.0,1762385.98,San Bernardino,1710139.0
1,2001.0,1888556.78,San Bernardino,1741416.0
2,2002.0,1931424.51,San Bernardino,1782268.0
3,2003.0,2107128.88,San Bernardino,1825379.0
4,2004.0,2235251.83,San Bernardino,1875063.0


### Tests that should pass before doing any EDA / modeling analysis

In [22]:
# check we have all CA counties
num_counties = 58
assert num_counties-1 == len(complete_feature_df.County.unique()) # minus Sutter bc it has no waste data

# check time range of our data is consistent (2000 - 2019) or depends on we decide on
assert all(complete_feature_df.Year < 2020)
assert all(complete_feature_df.Year > 1999)

# check waste data hasn't changed
assert np.isclose(1664783, get_feature(complete_feature_df, "Alameda", 2019, "Population"))
assert np.isclose(9956882, get_feature(complete_feature_df, "Los Angeles", 2012,  "Population"))
assert np.isclose(2846289, get_feature(complete_feature_df, "Orange", 2000, "Population"))

# check population data hasn't changed
assert np.isclose(1465264, get_feature(complete_feature_df, "Alameda", 2019, "Waste Produced (Tons)"))
assert np.isclose(8141712, get_feature(complete_feature_df, "Los Angeles", 2012, "Waste Produced (Tons)"))
assert np.isclose(3803041, get_feature(complete_feature_df, "Orange", 2000, "Waste Produced (Tons)"))

### Save complete feature df after passing tests

In [23]:
complete_feature_df.to_csv("../data/complete_feature_df.csv", index=False)